<a href="https://colab.research.google.com/github/Ossiumx/Previsao-de-Volume-de-madeira/blob/main/Otimiza%C3%A7%C3%A3o_XGBOOSTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, learning_curve, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Carregar dados
#dados = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/XGBoost_important_data.xlsx')

dados = pd.read_excel("XGB.xlsx")

In [ ]:
# Visualizar os dados
dados.info()

In [ ]:
# Separar as variáveis de entrada (X) e a variável de saída (y)
x = dados.drop(["RAIZVOL"], axis=1)
y = dados["RAIZVOL"]

In [ ]:
# Dividir os dados em conjunto de treinamento e conjunto de teste
x_train, x_separated, y_train, y_separated = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_separated, y_separated, test_size=0.5, random_state=42)

In [ ]:
# Definir o modelo
xgb_valmodel = xgb.XGBRegressor()
xgb_valpred = cross_val_predict(xgb_valmodel, x_train, y_train, cv=5)
xgb_valmape = np.mean(np.abs((y_train - xgb_valpred) / y_train)) * 100
xgb_valr2 = r2_score(y_train, xgb_valpred)
xgb_valmae = mean_absolute_error(y_train, xgb_valpred)
xgb_valrmse = np.sqrt(mean_squared_error(y_train, xgb_valpred))


In [ ]:
print("XGBoost:", "MAPE: {:.2f}%".format(xgb_valmape), "R²: {:.2F}".format(xgb_valr2), "RMSE: {:.4f}".format(xgb_valrmse), "MAE: {:.4f}".format(xgb_valmae))

In [ ]:
# Função para plotar curva de aprendizado para o mae

cv = 5

train_sizes=np.linspace(0.1, 1.0, 10)

def plot_learning_curve(model, x, y, cv, train_sizes):
    train_sizes, train_scores, test_scores = learning_curve(model, x, y, cv=cv, train_sizes=train_sizes, scoring='neg_mean_absolute_error')

    train_mean = -np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = -np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, 'o-', color='r', label='Treinamento')
    plt.plot(train_sizes, test_mean, 'o-', color='g', label='Validação')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')
    plt.xlabel('Tamanho do conjunto de treinamento')
    plt.ylabel('MAE')
    plt.title('Curva de Aprendizado para o MAE')
    plt.legend(loc='best')
    plt.show()

In [ ]:
# Avaliar a curva de aprendizado do modelo
xgb_model = xgb.XGBRegressor()
plot_learning_curve(xgb_model, x_train, y_train, cv, train_sizes)

In [ ]:
# Função para Otimizar hiperparâmetros baseado na curva de aprenziado para a métrica MAE

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1000, 6000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 150),
        "subsample": trial.suggest_float("subsample", 0.01, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 150),
        "random_state": 42,
    }
    xgb_model = xgb.XGBRegressor(**params)


    # Dividir os dados de treinamento em conjuntos de treinamento e validação
    x_train, x_separated, y_train, y_separated = train_test_split(x, y, test_size=0.3, random_state=42)
    x_test, x_val, y_test, y_val = train_test_split(x_separated, y_separated, test_size=0.5, random_state=42)

    xgb_model.fit(x_train, y_train)

    # Calcular os erros de treinamento e validação
    y_train_pred = xgb_model.predict(x_train)
    y_val_pred = xgb_model.predict(x_val)
    train_error = mean_absolute_error(y_train, y_train_pred)
    val_error = mean_absolute_error(y_val, y_val_pred)

    # Calcula a diferença entre os erros de treinamento e validação
    error_diff = abs(train_error - val_error)

    # Adicionar a diferença de erro ao MAE para criar uma métrica composta
    composite_metric = error_diff + mean_absolute_error(y_val, xgb_model.predict(x_val))

    return composite_metric

In [ ]:
# Otimizar hiperparâmetros
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=300)

In [ ]:
# Visualizar resultados
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 8910,learning_rate= 0.0061348615672835805, max_depth= 97, subsample= 0.6544117876474616, colsample_bytree= 0.8909681873163241, min_child_weight= 65) #0,30
#xgb_model = xgb.XGBRegressor(n_estimators = 1956,learning_rate= 0.017987036737857547, max_depth= 12, subsample= 0.7701567055152614, colsample_bytree= 0.9996729353742191, min_child_weight= 70) #0,31
#xgb_model = xgb.XGBRegressor(n_estimators = 3871,learning_rate= 0.01856876886536243, max_depth= 150, subsample= 0.8578916773331227, colsample_bytree= 0.8603549001123457, min_child_weight= 83)
xgb_model.fit(x_train, y_train)
y_pred_best = xgb_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred_best)
r2 = r2_score(y_test, y_pred_best)
mape = np.mean(np.abs((y_test - y_pred_best) / y_test)) * 100
mae = mean_absolute_error(y_test, y_pred_best)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_best))

print("XGBoost:", "MAPE: {:.2f}%".format(mape), "R²:", r2, "RMSE: {:.5f}".format(rmse), "MAE: {:.5f}".format(mae))

In [ ]:
# Avaliar curva de aprendizado para o ajuste final
plot_learning_curve(xgb_model, x_test, y_test, cv, train_sizes)